In [1]:
import pandas as pd
from my_functions import *

In [2]:
# create_pegmed()

In [3]:
# create_pegprod()

In [4]:
# create_all_clinical_tests_referrals_just_patid_and_dates_csv()
## this takes a long time to run!

In [5]:
# create_insomnia_entries()

In [6]:
# create_pt_features()

In [7]:
all_entries = pd.read_csv('data/pt_data/medcoded_clinical_entries.csv',delimiter=',')

In [8]:
add_sys_start_and_end_dates_plus_length_of_available_data_to_patient_features(all_entries)

In [11]:
pt_features = pd.read_csv('data/pt_data/pt_features.csv',delimiter=',')

In [12]:
pt_features

,patID,pracID,gender,birthyear,case_index,matchID,data_start,data_end,isCase,prediagnosis_data_length_in_days,sysdate_start,sysdate_end,length_of_data_pre_index_date,length_of_data_post_index_date
0,4923328,328,2,1916,2003-07-01,4355328,2000-01-01,2009-02-07,True,1277,2001-06-04,2005-05-16,757,685
1,1959216,216,1,1933,2011-02-25,6290216,2000-01-01,2012-06-01,True,4073,1995-12-20,2013-02-22,5546,728
2,9271420,420,1,1917,2003-06-20,2770420,2003-01-02,2009-12-29,True,169,1999-10-16,2004-05-26,1343,341
3,548429,429,2,1925,2005-05-11,5473429,2000-01-01,2012-06-01,True,1957,1999-11-29,2011-11-07,1990,2371
4,14185081,81,2,1917,2005-02-28,5951081,2000-01-01,2007-01-27,True,1885,1996-09-12,2006-02-16,3091,353
5,7329438,438,2,1941,2008-08-04,17742438,2000-01-01,2012-06-01,True,3138,1996-11-03,2013-05-09,4292,1739
6,6327373,373,1,1941,2009-03-26,5262373,2004-12-22,2010-09-10,True,1555,2001-12-11,2011-08-19,2662,876
7,5811024,24,1,1930,2010-12-14,6394024,2000-01-01,2012-06-01,True,4000,1997-10-31,2013-01-17,4792,765
8,6905371,371,1,1926,2006-09-19,15830371,2002-11-07,2009-07-22,True,1412,1999-03-12,2008-09-22,2748,734
9,7741629,629,1,1936,2004-06-10,9053629,2000-01-01,2012-06-01,True,1622,2001-12-06,2005-01-24,917,228
